In [ ]:
%pip  install farm-haystack[colab,ocr,preprocessing,file-conversion,metrics,inference]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 764.4/764.4 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.7/48.7 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 41.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 54.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.1/300.1 kB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 34.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 13.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━

In [ ]:
from haystack.document_stores import InMemoryDocumentStore

document_store = InMemoryDocumentStore()

In [ ]:
from haystack.nodes import EmbeddingRetriever

retriever = EmbeddingRetriever(
    document_store=document_store,
    embedding_model="sentence-transformers/all-MiniLM-L6-v2",
    use_gpu=True,
    scale_score=False,
)

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [ ]:
import pandas as pd
import json


# Get dataframe with columns "question", "answer" and some custom metadata
df = pd.read_json("faq_data.json")
# Minimal cleaning
df.fillna(value="", inplace=True)
df["question"] = df["question"].apply(lambda x: x.strip())
print(df.head())

# Create embeddings for our questions from the FAQs
# In contrast to most other search use cases, we don't create the embeddings here from the content of our documents,
# but rather from the additional text field "question" as we want to match "incoming question" <-> "stored question".
questions = list(df["question"].values)
df["embedding"] = retriever.embed_queries(queries=questions).tolist()
df = df.rename(columns={"question": "content"})

# Convert Dataframe to list of dicts and index them in our DocumentStore
docs_to_index = df.to_dict(orient="records")
document_store.write_documents(docs_to_index)

                                            question  \
0  Warum nennt man das erste Semester auch "Asses...   
1  Wo finde ich den Studienplan (zum ersten Semes...   
2  Wann finden die Klausurprüfungen zum Assessmen...   
3  Muss ich gleich zu allen Prüfungen des Assessm...   
4      Hilfe, ich habe eine Klausur nicht bestanden!   

                                              answer  
0  Das erste Semester soll Ihnen den Einstieg ins...  
1  Dies alles finden Sie in der Studien- und Prüf...  
2  Klausuren werden stets unmittelbar nach Ende d...  
3  Ja, Sie müssen in Ihrem ersten Semester an all...  
4  Falls Sie eine Klausur im ersten Anlauf nicht ...  


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from haystack.pipelines import FAQPipeline

faq_pipe = FAQPipeline(retriever=retriever)

In [ ]:
from haystack.pipelines import ExtractiveQAPipeline
from haystack.nodes import FARMReader

reader = FARMReader(model_name_or_path="deepset/gelectra-large-germanquad", use_gpu=True)
extract_pipe = ExtractiveQAPipeline(reader=reader , retriever=retriever)

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/359 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/240k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

# Combine FAQ Embedding Pipeline with ExtractivePipeline with LLM
if the prediction score is lower than 0.8, hence if the question is not ideally matched with an embedded question, the question is piped to the extractivePipeline which can use a specific retriever in combination with the generative capabilities of a chosen LLM

In [ ]:
from haystack import Pipeline
from haystack.pipelines import FAQPipeline, ExtractiveQAPipeline
from haystack.nodes import FARMReader

class CustomPipeline(Pipeline):
    def __init__(self):
        super().__init__()

        self.faq_pipeline = faq_pipe
        self.extractive_qa_pipeline = extract_pipe

    def run(self, query, **kwargs):
        # Use FAQPipeline first
        results = self.faq_pipeline.run(query=query, **kwargs)

        # Check if the score is below 0.8
        if results['answers'][0].score < 0.8:
            # If below 0.8, use ExtractiveQAPipeline
            return self.extractive_qa_pipeline.run(query=query, **kwargs)

        return results

In [ ]:
pipeline=CustomPipeline()

In [ ]:
from haystack.utils import print_answers

prediction= pipeline.run(query="Was ist das Assessmentsemester?")

print_answers(prediction, details="all")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Inferencing Samples: 100%|██████████| 1/1 [00:31<00:00, 31.81s/ Batches]

'Query: Was ist das Assessmentsemester?'
'Answers:'
[   <Answer {'answer': 'das erste Semester', 'type': 'extractive', 'score': 0.6472023725509644, 'context': 'Warum nennt man das erste Semester auch "Assessmentsemester"?', 'offsets_in_document': [{'start': 16, 'end': 34}], 'offsets_in_context': [{'start': 16, 'end': 34}], 'document_ids': ['283e29bd985ff966482960a5345b58a7'], 'meta': {'answer': 'Das erste Semester soll Ihnen den Einstieg ins Studium erleichtern und Ihnen helfen, frühzeitig einzuschätzen, ob der gewählte Studiengang Ihren Vorstellungen und Neigungen auch wirklich entspricht. Es ist so aufgebaut, dass Ihnen sowohl fachliche Grundlagen als auch Lern- und Arbeitstechniken, die für ein erfolgreiches Studium notwendig sind, vermittelt werden. Dadurch wird Ihre Studierfähigkeit gefördert, aber auch frühzeitig überprüft und bewertet.Das englische Wort "assessment" kann mit "Einschätzung" oder "Bewertung" übersetzt werden.Für das Assessmentsemester gibt es einige Besonderheiten

In [ ]:
from haystack.utils import print_answers

prediction= pipeline.run(query="Wie kann ich mich zurückstufen lassen?")

print_answers(prediction, details="all")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

'Query: Wie kann ich mich zurückstufen lassen?'
'Answers:'
[   <Answer {'answer': 'Im Hauptstudium ist eine freiwillige Rückstufung in das zuletzt durchlaufene Semester auf Antrag möglich. Voraussetzung dafür ist, dass Ihnen noch mindestens 4\xa0Prüfungsleistungen fehlen.Beachten Sie aber, dass nach einer Rückstufung keine Prüfungsleistungen aus höheren Semestern vorgezogen werden können!Der Antrag auf Rückstufung ist vor\xa0Beginn der ersten Vorlesungswoche ausgefüllt im Sekretariat des Prüfungsamtes abzugeben. Zuvor sollten Sie sich durch den/die Vorsitzenden des Prüfungsausschusses beraten lassen.Im Grundstudium ist eine Rückstufung ins 1. Semester nicht möglich. Eine Rückstufung ins 2. Semester erfolgt automatisch (s.o.) und ist ja meist auch nicht ganz freiwillig.\n', 'type': 'other', 'score': 0.89980554228533, 'context': 'Im Hauptstudium ist eine freiwillige Rückstufung in das zuletzt durchlaufene Semester auf Antrag möglich. Voraussetzung dafür ist, dass Ihnen noch mindestens 4\